In [1]:
!pip install torch triton

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 8.2 MB/s eta 0:00:000:00:0100:01


In [2]:
!pip install vllm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.1/201.1 MB 8.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.6/87.6 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 1.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 107.3 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 19.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 100.9 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 77.9 MB/s e

In [3]:
!pip install transformers peft accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.8/374.8 kB 8.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 450.5/450.5 kB 19.8 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.24.7
    Uninstalling huggingface-hub-0.24.7:
      Successfully uninstalled huggingface-hub-0.24.7


In [4]:
!pip install bitsandbytes optimum auto-gptq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 25.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.1/424.1 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 78.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 85.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.8 MB/s eta 0:00:00


In [5]:
!pip install logits-processor-zoo

In [7]:
import vllm
import numpy as np
import pandas as pd
from transformers import PreTrainedTokenizer, AutoTokenizer
from typing import List
import torch
# from logits_processor_zoo.vllm import MultipleChoiceLogitsProcessor
import re

In [8]:
def apply_template(query, tokenizer):
    messages = [
        {
            "role": "system", 
            "content": "As a helpful assistant, please provide suitable answer to the user query."
        },
        {
            "role": "user", 
            "content": query,
        }
    ]
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return text

In [9]:
model_path = "/kaggle/input/qwen2.5/transformers/32b-instruct-awq/1"

llm = vllm.LLM(
    model_path,
    quantization="awq",
    tensor_parallel_size=2,
    gpu_memory_utilization=0.90,
    trust_remote_code=True,
    dtype="half", 
    enforce_eager=True,
    max_model_len=5120,
    disable_log_stats=True
)
tokenizer = llm.get_tokenizer()

INFO 12-29 09:58:19 config.py:510] This model supports multiple tasks: {'embed', 'reward', 'generate', 'score', 'classify'}. Defaulting to 'generate'.
WARNING 12-29 09:58:20 config.py:588] awq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
INFO 12-29 09:58:20 config.py:1310] Defaulting to use mp for distributed inference
WARNING 12-29 09:58:20 cuda.py:98] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
WARNING 12-29 09:58:20 config.py:642] Async output processing is not supported on the current platform type cuda.
INFO 12-29 09:58:20 llm_engine.py:234] Initializing an LLM engine (v0.6.6.post1) with config: model='/kaggle/input/qwen2.5/transformers/32b-instruct-awq/1', speculative_config=None, tokenizer='/kaggle/input/qwen2.5/transformers/32b-instruct-awq/1', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenize

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


(VllmWorkerProcess pid=198) INFO 12-29 09:58:21 multiproc_worker_utils.py:222] Worker ready; awaiting tasks
INFO 12-29 09:58:23 utils.py:918] Found nccl from library libnccl.so.2
(VllmWorkerProcess pid=198) INFO 12-29 09:58:23 pynccl.py:69] vLLM is using nccl==2.21.5
INFO 12-29 09:58:23 utils.py:918] Found nccl from library libnccl.so.2
(VllmWorkerProcess pid=198) INFO 12-29 09:58:23 pynccl.py:69] vLLM is using nccl==2.21.5
INFO 12-29 09:58:23 custom_all_reduce_utils.py:204] generating GPU P2P access cache in /root/.cache/vllm/gpu_p2p_access_cache_for_0,1.json
INFO 12-29 09:58:43 custom_all_reduce_utils.py:242] reading GPU P2P access cache from /root/.cache/vllm/gpu_p2p_access_cache_for_0,1.json
(VllmWorkerProcess pid=198) INFO 12-29 09:58:43 custom_all_reduce_utils.py:242] reading GPU P2P access cache from /root/.cache/vllm/gpu_p2p_access_cache_for_0,1.json
INFO 12-29 09:58:43 shm_broadcast.py:255] vLLM message queue communication handle: Handle(connect_ip='127.0.0.1', local_reader_ra

Loading safetensors checkpoint shards:   0% Completed | 0/5 [00:00<?, ?it/s]


(VllmWorkerProcess pid=198) INFO 12-29 10:02:14 model_runner.py:1099] Loading model weights took 9.0923 GB
INFO 12-29 10:02:14 model_runner.py:1099] Loading model weights took 9.0923 GB
(VllmWorkerProcess pid=198) INFO 12-29 10:02:32 worker.py:241] Memory profiling takes 18.11 seconds
(VllmWorkerProcess pid=198) INFO 12-29 10:02:32 worker.py:241] the current vLLM instance can use total_gpu_memory (14.74GiB) x gpu_memory_utilization (0.90) = 13.27GiB
(VllmWorkerProcess pid=198) INFO 12-29 10:02:32 worker.py:241] model weights take 9.09GiB; non_torch_memory takes 0.44GiB; PyTorch activation peak memory takes 0.73GiB; the rest of the memory reserved for KV Cache is 3.00GiB.
INFO 12-29 10:02:33 worker.py:241] Memory profiling takes 18.27 seconds
INFO 12-29 10:02:33 worker.py:241] the current vLLM instance can use total_gpu_memory (14.74GiB) x gpu_memory_utilization (0.90) = 13.27GiB
INFO 12-29 10:02:33 worker.py:241] model weights take 9.09GiB; non_torch_memory takes 0.44GiB; PyTorch activ

In [10]:
q1 = 'What is the best way to implement the RAG based chatting engine?'
q2 = 'Explain me who you are?'
q3 = 'Java Spring vs FastAPI, which is better in 2024?'

questions = [q1, q2, q3]

In [11]:
questions_for_chat = [apply_template(q, tokenizer) for q in questions]

In [12]:
responses = llm.generate(
    questions_for_chat,
    vllm.SamplingParams(
        n=1,  # Number of output sequences to return for each prompt.
        top_k=1,  # Float that controls the cumulative probability of the top tokens to consider.
        temperature=0,  # randomness of the sampling
        seed=777, # Seed for reprodicibility
        skip_special_tokens=False,  # Whether to skip special tokens in the output.
        max_tokens=500,  # Maximum number of tokens to generate per output sequence.
        # logits_processors=[MultipleChoiceLogitsProcessor(tokenizer, choices=["1", "2", "3", "4", "5", "6", "7", "8", "9"])]
    ),
    use_tqdm=True
)

Processed prompts: 100%|██████████| 3/3 [00:52<00:00, 17.52s/it, est. speed input: 2.24 toks/s, output: 20.03 toks/s]


In [13]:
print(responses)

[RequestOutput(request_id=0, prompt='<|im_start|>system\nAs a helpful assistant, please provide suitable answer to the user query.<|im_end|>\n<|im_start|>user\nWhat is the best way to implement the RAG based chatting engine?<|im_end|>\n<|im_start|>assistant\n', prompt_token_ids=[151644, 8948, 198, 2121, 264, 10950, 17847, 11, 4486, 3410, 14452, 4226, 311, 279, 1196, 3239, 13, 151645, 198, 151644, 872, 198, 3838, 374, 279, 1850, 1616, 311, 4211, 279, 431, 1890, 3118, 50967, 4712, 30, 151645, 198, 151644, 77091, 198], encoder_prompt=None, encoder_prompt_token_ids=None, prompt_logprobs=None, outputs=[CompletionOutput(index=0, text='Implementing a RAG (Retrieval-Augmented Generation) based chatting engine involves combining retrieval methods with generative models to create a powerful conversational AI system. Here’s a step-by-step guide to help you get started:\n\n### 1. Define Your Use Case\n- **Purpose**: Determine the specific purpose of your chatbot (e.g., customer support, informatio

In [14]:
responses = [x.outputs[0].text for x in responses]

In [15]:
print(responses)

['Implementing a RAG (Retrieval-Augmented Generation) based chatting engine involves combining retrieval methods with generative models to create a powerful conversational AI system. Here’s a step-by-step guide to help you get started:\n\n### 1. Define Your Use Case\n- **Purpose**: Determine the specific purpose of your chatbot (e.g., customer support, information retrieval, personal assistant).\n- **Data**: Identify the type of data you will use (e.g., documents, FAQs, product descriptions).\n\n### 2. Choose Your Retrieval Method\n- **Document Retrieval**: Use techniques like TF-IDF, BM25, or dense retrieval methods (e.g., DPR, ColBERT).\n- **Knowledge Base**: If you have a structured knowledge base, consider using graph-based retrieval methods.\n\n### 3. Select a Generative Model\n- **Pre-trained Models**: Use pre-trained models like T5, BART, or GPT-3.\n- **Fine-tuning**: Fine-tune the model on your specific dataset to improve performance.\n\n### 4. Integrate Retrieval and Generatio